In [33]:
#imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

### PDE Setup

In [34]:
# Burger's equation: u_t + u*u_x = 0
def pde_res(model, x, t):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch([x, t])
        u = model(tf.concat([x, t], axis=1))
        u_x = tape.gradient(u, x)
        u_t = tape.gradient(u, t)
    del tape
    pde_residual = u_t + u*u_x
    
    return tf.reduce_mean(tf.square(pde_residual))

def u_0(x):
    return np.sin(x)  # u_0 = sin(x)


### Network Architecture

In [35]:
class PINN(tf.keras.Model):
    def __init__(self, layers):
        super(PINN, self).__init__()
        self.hidden_layers = []
        for units in layers:
            self.hidden_layers.append(tf.keras.layers.Dense(units, activation='tanh'))
        self.output_layer = tf.keras.layers.Dense(1)

    def call(self, x):
        z = x
        for layer in self.hidden_layers:
            z = layer(z)
        output = self.output_layer(z)
        return output
    
def to_tf_tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)


### Physics Inspired Loss and Training

In [36]:
def loss(model, x, t, u_true, pde_res):
    u_H = model(tf.concat([x, t], axis=1))
    data_loss = tf.reduce_mean(tf.square(u_H - u_true))
    
    pde_loss = pde_res(model, x, t)
    
    return data_loss + pde_loss


def train(model, x, t, u_true, pde_res, epochs, learning_rate):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            losss = loss(model, x, t, u_true, pde_res)
        gradients = tape.gradient(losss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        
        print(f"Epoch {epoch}, Loss: {losss.numpy()}")

### Training Data

In [37]:
def generate_data(num_x, num_t, xmin, xmax, tmin, tmax):
    x = np.linspace(xmin, xmax, num_x)
    t = np.linspace(tmin, tmax, num_t)
   
    X, T = np.meshgrid(x, t)
    X = X.flatten()[:, None]
    T = T.flatten()[:, None]
    
    U = u_0(X)  

    return to_tf_tensor(X), to_tf_tensor(T), to_tf_tensor(U)


# Parameters

layers = [128, 128, 128, 128]
learning_rate = 0.001
epochs = 512

x_points = 512
t_points = 200
xmin, xmax = -1, 1
tmin, tmax = 0, 1

### Training Model

In [38]:
model = PINN(layers)

x, t, u_true = generate_data(x_points, t_points, xmin, xmax, tmin, tmax)

train(model, x, t, u_true, lambda model, x, t: pde_res(model, x, t), epochs, learning_rate)


Epoch 0, Loss: 0.3337097465991974
Epoch 1, Loss: 0.14662709832191467
Epoch 2, Loss: 0.06681917607784271
Epoch 3, Loss: 0.16252253949642181
Epoch 4, Loss: 0.12553945183753967
Epoch 5, Loss: 0.08354584127664566
Epoch 6, Loss: 0.06569784879684448
Epoch 7, Loss: 0.07669568806886673
Epoch 8, Loss: 0.09336361289024353
Epoch 9, Loss: 0.09864979237318039
Epoch 10, Loss: 0.09599123150110245
Epoch 11, Loss: 0.0903782993555069
Epoch 12, Loss: 0.0810958668589592
Epoch 13, Loss: 0.06980000436306
Epoch 14, Loss: 0.0633203461766243
Epoch 15, Loss: 0.0644429475069046
Epoch 16, Loss: 0.06843984872102737
Epoch 17, Loss: 0.07229151576757431
Epoch 18, Loss: 0.07426580786705017
Epoch 19, Loss: 0.0706639289855957
Epoch 20, Loss: 0.06455948203802109
Epoch 21, Loss: 0.06138940155506134
Epoch 22, Loss: 0.060645196586847305
Epoch 23, Loss: 0.06088530644774437
Epoch 24, Loss: 0.062475256621837616
Epoch 25, Loss: 0.063908651471138
Epoch 26, Loss: 0.06339363753795624
Epoch 27, Loss: 0.06187715381383896
Epoch 28, L